In [ ]:
!pip install requests
!pip install Pillow
!pip install matplotlib
!pip install numpy
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step 1: Import Necessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from sklearn.model_selection import train_test_split

### Step 2: Load and Organize the Dataset

In [ ]:
import requests
import zipfile
import io

# Define the URL of the dataset zip file
dataset_url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/hxsnvwty3r-1.zip"

# Define the destination directory where you want to save and extract the dataset
destination_dir = "/content/MangoLeafBD_Dataset"

# Create the destination directory if it doesn't exist
!mkdir -p {destination_dir}

# Download the dataset zip file
response = requests.get(dataset_url)
dataset_zip = zipfile.ZipFile(io.BytesIO(response.content))

# Extract the dataset to the destination directory
dataset_zip.extractall(destination_dir)

# Close the zip file
dataset_zip.close()

print("Dataset downloaded and extracted successfully.")

Dataset downloaded and extracted successfully.


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the root dataset directory
root_dir = "/content/MangoLeafBD_Dataset/MangoLeafBD Dataset"

# Define the target train and validation directories
train_dir = "/content/MangoLeafBD_Train"
val_dir = "/content/MangoLeafBD_Val"

# Create train and validation directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Define the class names
class_names = [
    "Anthracnose", "Bacterial Canker", "Cutting Weevil", "Die Back",
    "Gall Midge", "Healthy", "Powdery Mildew", "Sooty Mould"
]

# Define the train-validation split ratio (adjust as needed)
val_split = 0.2

# Iterate through class names and move images to train and validation folders
for class_name in class_names:
    class_dir = os.path.join(root_dir, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)

    # Create class directories in train and validation folders
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    # List all images in the class directory
    images = os.listdir(class_dir)

    # Split images into train and validation sets
    train_images, val_images = train_test_split(images, test_size=val_split, random_state=42)

    # Move images to train and validation folders
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_class_dir, image)
        shutil.move(src, dst)

    for image in val_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(val_class_dir, image)
        shutil.move(src, dst)

print("Dataset organized into train and validation folders.")

Dataset organized into train and validation folders.


### Step 3: Data Augmentation

In [ ]:
# Define image preprocessing parameters
batch_size = 32
image_size = (224, 224)

# Create data generators with data augmentation for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 3200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


### Step 4: Build a Complex CNN Model

In [ ]:
model = Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the output and add fully connected layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')  # 8 classes for your dataset
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Step 5: Train the Model

In [ ]:
# Define the number of epochs
epochs = 20

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

Epoch 1/20
100/100 [==============================] - 63s 480ms/step - loss: 1.8318 - accuracy: 0.2559 - val_loss: 1.4914 - val_accuracy: 0.4238
Epoch 2/20
100/100 [==============================] - 50s 501ms/step - loss: 1.2359 - accuracy: 0.5288 - val_loss: 0.7422 - val_accuracy: 0.7300
Epoch 3/20
100/100 [==============================] - 53s 529ms/step - loss: 0.8498 - accuracy: 0.6891 - val_loss: 0.5606 - val_accuracy: 0.7713
Epoch 4/20
100/100 [==============================] - 49s 495ms/step - loss: 0.5974 - accuracy: 0.7925 - val_loss: 0.3965 - val_accuracy: 0.8700
Epoch 5/20
100/100 [==============================] - 51s 506ms/step - loss: 0.5244 - accuracy: 0.8106 - val_loss: 0.2462 - val_accuracy: 0.9237
Epoch 6/20
100/100 [==============================] - 48s 484ms/step - loss: 0.3586 - accuracy: 0.8703 - val_loss: 0.4718 - val_accuracy: 0.8288
Epoch 7/20
100/100 [==============================] - 48s 477ms/step - loss: 0.3859 - accuracy: 0.8637 - val_loss: 0.2203 - val_ac

### Step 6: Evaluate the Model

In [ ]:
# Evaluate the model on the training data
train_metrics = model.evaluate(train_generator)

# Evaluate the model on the validation data
val_metrics = model.evaluate(val_generator)

# Print the metrics for the training set
print("Metrics for Training Set:")
print(f"Loss: {train_metrics[0]}")
print(f"Accuracy: {train_metrics[1]}")

# Print the metrics for the validation set
print("\nMetrics for Validation Set:")
print(f"Loss: {val_metrics[0]}")
print(f"Accuracy: {val_metrics[1]}")

25/25 [==============================] - 1s 54ms/step - loss: 0.0587 - accuracy: 0.9825
Metrics for Training Set:
Loss: 0.06675581634044647
Accuracy: 0.9743750095367432

Metrics for Validation Set:
Loss: 0.05869104340672493
Accuracy: 0.9825000166893005


### Step 7: Save the Model

In [ ]:
# Save the trained model
model.save("mango_leaf_disease_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Step 8: Make Predictions

8.1: Load the Trained Model

  First, load the saved model you trained earlier. Make sure you have the correct path to the saved model (e.g., "mango_leaf_disease_model.h5").

In [ ]:
# Load the trained model
loaded_model = keras.models.load_model("mango_leaf_disease_model.h5")

8.2: Preprocess New Images

Before making predictions, you need to preprocess the new images in the same way you preprocessed the training and validation data.

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the path to the new image you want to test
new_image_path = "/path/to/your/new/image.jpg"

# Load and preprocess the new image
img = image.load_img(new_image_path, target_size=image_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize pixel values

FileNotFoundError: ignored

8.3: Make Predictions

Now, you can use the loaded model to make predictions on the preprocessed image:

In [ ]:
# Make predictions
predictions = loaded_model.predict(img)

# Get class labels and corresponding probabilities
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
predicted_class = class_labels[np.argmax(predictions)]

# Print the result
print(f"The predicted class is: {predicted_class}")
print(f"Prediction probabilities: {predictions[0]}")

### Step 9: Save the Model to Google Drive
To save the trained model to Google Drive, follow these steps:

9.1: Specify Source and Destination Paths

Specify the source path, which is the path to your saved model, and the destination path in your Google Drive where you want to save the model.

In [ ]:
source_path = 'mango_leaf_disease_model.h5'  # Your model file
destination_path = '/content/drive/MyDrive/'  # Destination in your Google Drive

9.2: Copy the Model to Google Drive


Use the shutil library to copy the model from your Colab environment to Google Drive:

In [ ]:
import shutil

# Copy the model to Google Drive
shutil.copy(source_path, destination_path)

'/content/drive/MyDrive/mango_leaf_disease_model.h5'